In [ ]:
import torch


In [3]:
from torch import nn

net=nn.Sequential(nn.Linear(20,256),
                  nn.ReLU(),
                  nn.Linear(256,10))
x=torch.rand([2,20],requires_grad=True)
net(x)

tensor([[ 0.0024, -0.0594, -0.0255, -0.0598,  0.0987, -0.1932, -0.0369, -0.1168,
          0.0750,  0.2130],
        [-0.1057, -0.0136, -0.0291,  0.0695,  0.0243, -0.1099, -0.0600, -0.0006,
          0.1379,  0.0697]], grad_fn=<AddmmBackward0>)

In [13]:
from torch.nn import functional as F
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))
net=MLP()
net(x)

tensor([[-0.2295, -0.0390,  0.0836, -0.2754,  0.0601,  0.0010,  0.0933,  0.0803,
          0.1384, -0.1377],
        [-0.1802, -0.0792,  0.1049, -0.2592,  0.0502,  0.1159,  0.0770,  0.0852,
          0.0522, -0.1805]], grad_fn=<AddmmBackward0>)

In [19]:
class FixMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand([20,20],requires_grad=True)
        self.linear=nn.Linear(20,20)
    def forward(self,x):
        x=self.linear(x)
        x=F.relu(torch.mm(x,self.rand_weight)+1)
        x=self.linear(x)
        while x.abs().sum()>1:
            x/=2
        return x.sum()
net=FixMLP()
net(x)

tensor(-0.0320, grad_fn=<SumBackward0>)

In [20]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0495],
        [-0.0820]], grad_fn=<AddmmBackward0>)

In [21]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1971,  0.1875,  0.3057, -0.3244, -0.1922,  0.1017,  0.2911, -0.3439]])), ('bias', tensor([-0.2181]))])


In [22]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2181], requires_grad=True)
tensor([-0.2181])


In [28]:
print([(i,k) for i,k in net.named_parameters()])

[('0.weight', Parameter containing:
tensor([[-0.1515, -0.2626,  0.3383,  0.4945],
        [ 0.4485, -0.2808, -0.0754,  0.3334],
        [ 0.2859,  0.3076,  0.0753,  0.1680],
        [-0.4506, -0.4266,  0.4324, -0.4394],
        [-0.1500, -0.1373, -0.0658, -0.3953],
        [ 0.1940,  0.1810,  0.0532,  0.4866],
        [-0.3189, -0.3646,  0.3632,  0.0605],
        [-0.0617,  0.0129, -0.4018,  0.0296]], requires_grad=True)), ('0.bias', Parameter containing:
tensor([-0.0048,  0.1358, -0.1158, -0.2165,  0.0749, -0.3402, -0.4369,  0.2469],
       requires_grad=True)), ('2.weight', Parameter containing:
tensor([[-0.1971,  0.1875,  0.3057, -0.3244, -0.1922,  0.1017,  0.2911, -0.3439]],
       requires_grad=True)), ('2.bias', Parameter containing:
tensor([-0.2181], requires_grad=True))]


In [55]:
share=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                  share,nn.ReLU(),
                  share,nn.ReLU(),
                  nn.Linear(8,1))
x=torch.rand(1,4)
net(x)

tensor([[0.2745]], grad_fn=<AddmmBackward0>)

In [56]:
!nvidia-smi

Wed Mar 15 20:52:56 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti    WDDM | 00000000:01:00.0  On |                  Off |
|  0%   40C    P8               27W / 450W|   1747MiB / 24564MiB |     14%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [58]:
torch.device('cuda:1')

device(type='cuda', index=1)

In [61]:
torch.cuda.device_count()

1

In [65]:
x=torch.rand(1,5,device='cuda')
x.device

device(type='cuda', index=0)

In [67]:
net=nn.Sequential(nn.Linear(5,1))
net.to(device='cuda')
net(x)

tensor([[0.4219]], device='cuda:0', grad_fn=<AddmmBackward0>)